In [1]:
import os
import requests
import json
import re
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import datetime
from pyspark.sql.functions import regexp_replace

In [2]:
LANDED = './landed/'

In [3]:
builder = SparkSession.builder.appName("Extractor API")
builder.config(
    "spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2")
builder.config("spark.speculation", "false")
builder.config("spark.sql.parquet.compression.codec", "gzip")
builder.config("spark.debug.maxToStringFields", "100")
builder.config("spark.driver.memory", "2g")
builder.config("spark.driver.cores", "1")
builder.config("spark.executor-memory", "6g")
builder.config("spark.executor.cores", "5")
builder.master("local[*]")

spark = builder.getOrCreate()
spark

In [4]:
request_token = '<TOKEN>'
request_base_url = '<REQUEST_BASE_END_POINT>'
endpoint_getProjects = 'getProjectWithNewApi'
endpoint_getReports = 'getReports'

In [5]:
data = {'token': request_token }
response = requests.post(request_base_url+endpoint_getProjects, data) 

In [6]:
listOfProjects = json.loads(response.text)

In [7]:
def is_json(myjson):
  try:
    json_object = json.loads(myjson)
  except ValueError as e:
    return False
  return True

In [8]:
def get_files_to_download(url):
    data = {'token': '<TOKEN_2>'}
    url = url.replace('https://','').replace('www.','')
    
    url = 'http://'+url
    retorno_ws = requests.post(url, data)
    files_to_download = []
    if is_json(retorno_ws.text):
        
        retorno = json.loads(retorno_ws.text)
    
        base_url = re.sub('/api/.*', '', url)
        if 'type' in retorno:
            if retorno['type'] == 'download':
                if 'files' in retorno:
                    files_to_download = [base_url+file for file in retorno['files']]
    return files_to_download


In [9]:
def building_endpoint(url):
    models = ['aluno', 'user', 'pedido', 'matricula', 'curso', 'transacao_pagseguro', 'log_aula', 'logIzz']
    urls = [url + '/api/extract/' + x for x in models]
    return urls

In [10]:
def download_files(url):
    domain = url.replace('https://','').replace('www.','').replace('http://', '')
    domain = re.sub('/files/.*', '', domain)
    filename = re.sub('.*/etx/', '', url)
    
    folder_path = LANDED + domain
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
        
    download = requests.get(url)
    with open(folder_path+'/'+filename,'wb') as f: 
        f.write(download.content)
    

In [11]:
endpoints = spark.sparkContext.parallelize(listOfProjects).flatMap(building_endpoint)

In [12]:
endpoints_array = endpoints.collect()

In [13]:
files_to_download = spark.sparkContext.parallelize(endpoints_array).flatMap(get_files_to_download).collect()

In [14]:
download_now = spark.sparkContext.parallelize(files_to_download).foreach(download_files)